In [1]:
import utilities as u
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader

In [2]:
file_range = (0,600)

# Define Dataset

In [3]:
class TopLevelProofDataset(InMemoryDataset):
    def __init__(self, root='', transform=None, pre_transform=None):
        super(TopLevelProofDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['../top_level_proofs.dataset']
    
    def download(self):
        pass
    
    def process(self):
        data_list = []
        
        for thm, y in data:
            thm = u.process_theorem(thm)
            tree, distinct_features = u.thm_to_tree(thm)
            
            x, edge_index = u.graph_to_data(tree, list(distinct_features))
            
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# SAGEConv Layer

In [4]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='max') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        print('b1')
        edge_index, _ = remove_self_loops(edge_index)
        print('b2')
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        print('b3')
        
        print(x.size(0))
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        print('b4')
        x_j = self.lin(x_j)
        print('b5')
        x_j = self.act(x_j)
        print('b6')
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

# GNN definition

In [5]:
embed_dim = 115
from torch_geometric.nn import TopKPooling, GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GCNConv(dataset.num_node_features, 128)
        self.embedding = torch.nn.Embedding(num_embeddings=df.item_id.max() +1, embedding_dim=embed_dim)
#         self.pool1 = TopKPooling(128, ratio=0.8)
#         self.conv2 = SAGEConv(128, 128)
#         self.pool2 = TopKPooling(128, ratio=0.8)
#         self.conv3 = SAGEConv(128, 128)
#         self.pool3 = TopKPooling(128, ratio=0.8)
#         self.lin1 = torch.nn.Linear(256, 128)
#         self.lin2 = torch.nn.Linear(128, 64)
#         self.lin3 = torch.nn.Linear(64, 1)
#         self.bn1 = torch.nn.BatchNorm1d(128)
#         self.bn2 = torch.nn.BatchNorm1d(64)
#         self.act1 = torch.nn.ReLU()
#         self.act2 = torch.nn.ReLU()        
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = x.squeeze(1)        

        x = self.conv1(x, edge_index)
        x = F.relu(x)

#         x, edge_index, _, batch, _ = self.pool1(x, edge_index, None, batch)
#         x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

#         x = F.relu(self.conv2(x, edge_index))
     
#         x, edge_index, _, batch, _ = self.pool2(x, edge_index, None, batch)
#         x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

#         x = F.relu(self.conv3(x, edge_index))

#         x, edge_index, _, batch, _ = self.pool3(x, edge_index, None, batch)
#         x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

#         x = x1 + x2 + x3

#         x = self.lin1(x)
#         x = self.act1(x)
#         x = self.lin2(x)
#         x = self.act2(x)      
#         x = F.dropout(x, p=0.5, training=self.training)

#         x = torch.sigmoid(self.lin3(x)).squeeze(1)

        return x

# Data inspections

In [6]:
data = u.make_data()
#data = data[0:2]

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590


In [8]:
from tqdm import tqdm
distinct_features = set()

for (thm, _) in tqdm(data):
#     if idx % 1000 == 0:
#         print(f'{idx} / {len(data)}')
    thm = u.process_theorem(thm)
    thm_tree, features = u.thm_to_tree(thm)
    distinct_features = distinct_features.union(features)
print(len(distinct_features))

100%|██████████| 13677/13677 [02:02<00:00, 111.57it/s]

878


In [35]:
test_thm = '(fun (a A B) (a A (a A B)))'
print(test_thm)
thm = u.process_theorem(test_thm)
print(thm)
thm_tree = u.thm_to_tree(thm)
print(thm_tree)

(fun (a A B) (a A (a A B)))
['(', 'fun', '(', 'a', 'A', 'B', ')', '(', 'a', 'A', '(', 'a', 'A', 'B', ')', ')', ')']
(<utilities.data_structures.Tree object at 0x7f41e66b6c50>, {'fun', 'a', 'A', 'B'})


In [36]:
counter = dict()
for _, y in data:
    if y in counter:
        counter[y] += 1
    else:
        counter[y] = 1
counter = list(counter.items())
counter.sort(key=lambda x: x[1], reverse=True)
percentages = [(x, y/len(datapoints)*100) for x,y in counter]
percentages

NameError: name 'data' is not defined

# Create Dataset

In [37]:
dataset = TopLevelProofDataset()
print(dataset)
loader = DataLoader(dataset, batch_size=1, shuffle=True)
for batch in loader:
    print(batch)

TopLevelProofDataset(2)
Batch(batch=[115], edge_index=[2, 228], x=[115, 1], y=[1])
Batch(batch=[43], edge_index=[2, 84], x=[43, 1], y=[1])


# Train

In [50]:
batch_size = 1
num_epochs = 1

def train():
    model.train()

    loss_all = 0
    for data in train_loader:
        print(data.x.shape)
        x = data.x.squeeze(1)
        print(data.edge_index.shape)
        g = GCNConv(115, 128)
        g(x, data.edge_index)
        

        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        optimizer.step()
    return loss_all / len(train_dataset)
    
device = torch.device('cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
crit = torch.nn.BCELoss()
train_loader = DataLoader(dataset, batch_size=batch_size)
for epoch in range(num_epochs):
    train()

torch.Size([115, 1])
torch.Size([2, 228])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)